In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import multiprocessing
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from common import Common, Timer
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import random

pd.set_option('display.max_colwidth', -1)

C:\Users\bill\software\anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [3]:
PATH = 'C:/Users/bill/Documents/projects/data/kaggle/complaints'
data = pd.read_csv(os.path.join(PATH, 'consumer_complaints.csv'), low_memory=False)
data = data[~data['consumer_complaint_narrative'].isna()].reset_index()
data = data[['product', 'consumer_complaint_narrative']].rename(columns={
    'consumer_complaint_narrative': 'complaints'
})

In [4]:
data.head()

,product,complaints
0,Debt collection,XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n
1,Consumer Loan,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX XXXX 2015, there was a delinquent balance of {$1400.00}. The monies from the GAP insurance and my insurance company has been paid, M & T Bank says that I still owe {$620.00}, of which {$210.00} has already been paid. I contacted M & T Bank via return receipt mail, but I have not been given the courtesy of a response yet. \n"
2,Mortgage,"In XX/XX/XXXX my wages that I earned at my job decreased by almost half, by XX/XX/XXXX I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. \nIn early XX/XX/XXXX I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called XXXX. They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from XXXX and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork began to be processed for some kind of approval. In XX/XX/XXXX I was in a trial period of 3 months of payments based on a slightly modified amount. Roughly {$75.00} less than what I was paying on my original payment. However, my caseworker with WFB, failed to tell me that since the payments during this time were not the full amount of the original mortgage payment so they were not applied to the loan. I discovered this in XX/XX/XXXX when I was told by a creditor with XXXX that she could n't restore my line of credit because of something to do with my mortgage. I then called and found this out. My caseworker at WFB then told me it was n't her responsibility to tell me this information. I told her that she could have told me that if I could come up with the remaining amount of {$75.00} then the payments would be applied. Instead, she chose to withhold this information and damage my credit even further. Now we are into 5 months ( including the 90 days WFB told me I had to fall behind until they would help me ) without an applied payment to my home loan, going on 6 months. \nMore paperwork was lost in XXXX as the modification was being finalized and my loan modification did n't go through until XX/XX/XXXX. I spoke to a lawyer that is working on a class-action lawsuit against WFB for people in a s

In [4]:
MAX_WORKERS = multiprocessing.cpu_count() - 1
B = 16
E = 10
T = 500
Y = len(data['product'].unique())

In [6]:
class Model(nn.Module):
    
    #model = 'distilbert-base-cased'
    def __init__(self, model, T, Y, device):
        super(Model, self).__init__()
        self.T = T
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model, do_lower_case=True)
        self.model = AutoModel.from_pretrained(model)
        self.linear = nn.Linear(768, Y)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, inputs):
        tokens = self.tokenizer.batch_encode_plus(
            inputs, 
            add_special_tokens=True, 
            return_tensors='pt', 
            pad_to_max_length=True
        )['input_ids'].to(self.device)
        output = self.model(tokens)[0][:,0,:]
        output = self.dropout(output)
        output = self.linear(output)
        return output

In [7]:
device = Common.device()
cp = Common.load_checkpoint('./models/model.tar')
model = Model('distilbert-base-uncased', T, Y, device).to(device)
model.load_state_dict(cp['state_dict'])

<All keys matched successfully>